In [ ]:
# Import python packages
import streamlit as st
import numpy as np
import pandas as pd
from snowflake.snowpark.functions import(
col, count, avg, max as max_, min as min_, dateadd, current_date, lit, sum as sum_, 
coalesce, datediff, any_value, when, array_unique_agg, array_size, regexp_replace, iff, nullifzero
)
from snowflake.ml.feature_store import FeatureStore, Entity, FeatureView, CreationMode
# We can also use Snowpark for our analyses!
from snowflake.snowpark.context import get_active_session
session = get_active_session()

## Data preparation

#### Ads

In [ ]:
user_features_ads_df = session.sql("""
WITH initial_table as (
  SELECT
    date_utc,
    user_id_hex,
    SPLIT_PART("payload.ad_event_data.ad_unit_id", '_', -1) || '' AS feature_name,
    count(*) AS feature_value_numeric
FROM dev.data_science.adshoweffective
    WHERE user_id_hex IS NOT NULL
    AND date_utc is not null 
    -- AND date_utc > '2025-11-23'
GROUP BY ALL
)
, pivot_table as (
SELECT *
FROM initial_table
PIVOT (
    MAX(feature_value_numeric) FOR feature_name IN (
    'conversationendinterstitial' as conversationendinterstitial
    , 'p1smsrichmedia' as p1smsrichmedia
    ,'postcallnativevideo' postcallnativevideo
    , 'callendinterstitial' callendinterstitial
    , 'unifiedinstreamlarge' unifiedinstreamlarge
    ,'p1ad' p1ad
    , 'unifiedinstreamsmall' unifiedinstreamsmall
    , 'banner' banner
    , 'callscreensmall' callscreensmall
    , 'keyboardmedrect' keyboardmedrect
    , 'iaprewardedvideo' iaprewardedvideo
    , 'p1smsnative' p1smsnative
    , 'textinstreamsmall' textinstreamsmall
    , 'textinstreamlarge' textinstreamlarge
    , 'tabletmedrect' tabletmedrect
    , 'callscreenlarge' callscreenlarge
    , 'rewardedvideo' rewardedvideo)
)
)
select date_utc
, user_id_hex
, COALESCE(conversationendinterstitial, 0) as conversationendinterstitial
, COALESCE(p1smsrichmedia, 0) as p1smsrichmedia
, COALESCE(postcallnativevideo, 0) as postcallnativevideo
, COALESCE(callendinterstitial, 0) as callendinterstitial
, COALESCE(unifiedinstreamlarge, 0) as unifiedinstreamlarge
, COALESCE(p1ad, 0) as p1ad
, COALESCE(unifiedinstreamsmall, 0) as unifiedinstreamsmall
, COALESCE(banner, 0) as banner
, COALESCE(callscreensmall, 0) as callscreensmall
, COALESCE(keyboardmedrect, 0) as keyboardmedrect
, COALESCE(iaprewardedvideo, 0) as iaprewardedvideo
, COALESCE(p1smsnative, 0) as p1smsnative
, COALESCE(textinstreamsmall, 0) as textinstreamsmall
, COALESCE(textinstreamlarge, 0) as textinstreamlarge
, COALESCE(tabletmedrect, 0) as tabletmedrect
, COALESCE(callscreenlarge, 0) as callscreenlarge
, COALESCE(rewardedvideo, 0) as rewardedvideo
-- , 'TEST' as test_column
from pivot_table
""")
# user_features_ads_df.show()

#### Call quality

In [ ]:
user_features_call_quality_df = session.sql("""
WITH initial_table as (
SELECT
    date_utc,
    USER_ID_HEX,
    COUNT_IF(COALESCE(num_bad_mos_periods, 0) > 0)::FLOAT AS calls_with_bad_mos,
    AVG(computed_mos)::FLOAT AS average_mos,
    MAX(RTP_SETUP_TIME)::FLOAT AS max_rtp_setup_time
    , count(*)  as dup_check
FROM
    dev.data_science.legacy_call_end
WHERE
    1=1
    -- AND date_utc >= '2025-11-01'
    AND leg_a_call_id not like 'no_active_calls%'
    AND lower(call_direction) in ('inbound', 'outbound')
    AND call_duration > 0
    AND user_id_hex <> '000-00-000-000000000'
GROUP BY ALL
)

select date_utc
    , user_id_hex
    , COALESCE(calls_with_bad_mos, 0) calls_with_bad_mos
    , COALESCE(average_mos, 0) average_mos
    , COALESCE(max_rtp_setup_time, 0) max_rtp_setup_time
FROM initial_table
""")
# user_features_call_quality_df.show()

#### Call rating

In [ ]:
user_features_call_rating_df =session.sql("""
WITH initial_table as (
SELECT
    date_utc,
    user_id_hex,
    COUNT(call_rating)::FLOAT AS call_rating_count,
    AVG(call_rating)::FLOAT AS avg_call_rating,
    MAX(call_rating)::FLOAT AS max_call_rating,
    MIN(call_rating)::FLOAT AS min_call_rating,
FROM
    dev.data_science.call_ratings_combined_sources
WHERE
    call_rating > 0
    AND user_id_hex <> '000-00-000-000000000'
GROUP BY ALL
)
select 
    date_utc
    , user_id_hex
    , COALESCE(call_rating_count, 0) call_rating_count
    , COALESCE(avg_call_rating, 0) avg_call_rating
    , COALESCE(max_call_rating, 0) max_call_rating
    , COALESCE(min_call_rating, 0) min_call_rating
FROM initial_table
""")
# user_features_call_rating_df.show()

#### Data usage

In [ ]:
user_features_data_usage_df = session.sql("""
WITH initial_table as (
SELECT 
    c.date_utc,
    up.user_id_hex,
    SUM(c.mb_usage) AS mb_usage
FROM dev.data_science.cost_user_daily_tmobile_cost c
JOIN dev.data_science.user_profiles up ON c.username = up.latest_username
WHERE
    up.user_id_hex <> '000-00-000-000000000'
GROUP BY ALL

)
SELECT 
    date_utc
    , user_id_hex
    , COALESCE(mb_usage, 0) as mb_usage
FROM initial_table
""")
# user_features_data_usage_df.show()

#### drawer event

In [ ]:
user_features_drawer_event_df = session.sql("""
WITH initial_tbl as ( 
SELECT
    date_utc,
    user_id_hex,
    '' || "payload.properties.DrawerEvent" AS feature_name,
    COUNT(*) AS feature_value_numeric
FROM dev.data_science.PROPERTYMAP pp
WHERE
    "payload.properties.DrawerEvent" IS NOT NULL
    -- AND DATE_UTC >= '2025-11-10'
GROUP BY ALL
)
, pivot_table as (
SELECT *
FROM initial_tbl
PIVOT (
    MAX(feature_value_numeric) FOR feature_name IN (
    'ConversationsTapped' as ConversationsTapped,
    'Use TextNow without WifiTapped' as UseTextNowwithoutWifiTapped
    ,'ContactsTapped' ContactsTapped
    ,'AdFreePlusTapped' AdFreePlusTapped
    ,'CallHistoryTapped' CallHistoryTapped
    ,'My WalletTapped' My_WalletTapped
    ,'AdFreeLiteTapped' AdFreeLiteTapped
    ,'WorldNewsTapped' WorldNewsTapped
    ,'MyWalletTapped' as MyWalletTapped
    ,'Earn Credits & RewardsTapped' EarnCreditsRewardsTapped
    ,'BlogTapped' BloggedTapped
    ,'TextNowNewsTapped' TNNewsTapped
    ,'Activate your SIM CardTapped' ActivateSimCardTapped
    ,'ShareAppTapped' ShareAppTapped
    ,'Use TextNow Without Wi-Fi Tapped' TNwoWiFiTapped
    ,'EarnCreditsTapped' EarnCreditsTapped
    ,'Close' CloseTapped
    ,'My StoreTapped' MyStoreTapped
    ,'MyOffersTapped' MyOffersTapped
    ,'TextNow PerksTapped' TNPerksTapped
    ,'SupportTapped' SupportTapped
    ,'Get Free StuffTapped' GetFreeStuffTapped
    ,'Open' OpenTapped
    ,'Remove Ads & Loc NumberTapped' RemoveAdsLocNumTapped
    ,'ActiveSIMTapped' ActiveSIMTapped
    ,'ShareNumberTapped' ShareNumberTapped
    ,'Priority Support EmailTapped' PriorityEmailTapped
    ,'TextNowPhonesTapped' TNPhonesTapped
    ,'Use TextNow without WiFiTapped'   UseTNwoWifiTapped
    ,'SettingsTapped' SettingsTapped
    ,'Get High-Speed DataTapped' HighSpeedDataTapped
    ,'RemoveAdsTapped' RemoveAdsTapped
    ,'FreeCoverageTapped' FreeCoverageTapped
    ,'Use TextNow Without Wi-FiTapped' UTNwoWIFITapped)
)
)
select date_utc
, user_id_hex ,
COALESCE(ConversationsTapped, 0) AS ConversationsTapped,
COALESCE(UseTextNowwithoutWifiTapped, 0) AS UseTextNowwithoutWifiTapped,
COALESCE(ContactsTapped, 0) AS ContactsTapped,
COALESCE(AdFreePlusTapped, 0) AS AdFreePlusTapped,
COALESCE(CallHistoryTapped, 0) AS CallHistoryTapped,
COALESCE(My_WalletTapped, 0) AS My_WalletTapped,
COALESCE(AdFreeLiteTapped, 0) AS AdFreeLiteTapped,
COALESCE(WorldNewsTapped, 0) AS WorldNewsTapped,
COALESCE(MyWalletTapped, 0) AS MyWalletTapped,
COALESCE(EarnCreditsRewardsTapped, 0) AS EarnCreditsRewardsTapped,
COALESCE(BloggedTapped, 0) AS BloggedTapped,
COALESCE(TNNewsTapped, 0) AS TNNewsTapped,
COALESCE(ActivateSimCardTapped, 0) AS ActivateSimCardTapped,
COALESCE(ShareAppTapped, 0) AS ShareAppTapped,
COALESCE(TNwoWiFiTapped, 0) AS TNwoWiFiTapped,
COALESCE(EarnCreditsTapped, 0) AS EarnCreditsTapped,
COALESCE(CloseTapped, 0) AS CloseTapped,
COALESCE(MyStoreTapped, 0) AS MyStoreTapped,
COALESCE(MyOffersTapped, 0) AS MyOffersTapped,
COALESCE(TNPerksTapped, 0) AS TNPerksTapped,
COALESCE(SupportTapped, 0) AS SupportTapped,
COALESCE(GetFreeStuffTapped, 0) AS GetFreeStuffTapped,
COALESCE(OpenTapped, 0) AS OpenTapped,
COALESCE(RemoveAdsLocNumTapped, 0) AS RemoveAdsLocNumTapped,
COALESCE(ActiveSIMTapped, 0) AS ActiveSIMTapped,
COALESCE(ShareNumberTapped, 0) AS ShareNumberTapped,
COALESCE(PriorityEmailTapped, 0) AS PriorityEmailTapped,
COALESCE(TNPhonesTapped, 0) AS TNPhonesTapped,
COALESCE(UseTNwoWifiTapped, 0) AS UseTNwoWifiTapped,
COALESCE(SettingsTapped, 0) AS SettingsTapped,
COALESCE(HighSpeedDataTapped, 0) AS HighSpeedDataTapped,
COALESCE(RemoveAdsTapped, 0) AS RemoveAdsTapped,
COALESCE(FreeCoverageTapped, 0) AS FreeCoverageTapped,
COALESCE(UTNwoWIFITapped, 0) AS UTNwoWIFITapped
from pivot_table
""")
# user_features_drawer_event_df.show()

#### inbound calls

In [ ]:
user_features_inbound_calls_df = session.sql("""
WITH initial_table as (

SELECT
    ic.date_utc,
    up.user_id_hex,
    COUNT(1)::DECIMAL(10,2) as inbound_count,
    SUM(ic.call_secs::DECIMAL(10,2)) as inbound_duration_sec,        
    NULLIFZERO(count_if(ic.call_secs = 0))::DECIMAL(10,2) as inbound_unanswered,
    NULLIFZERO(count_if(ic.call_secs > 15))::DECIMAL(10,2) as inbound_count_over_15s,
    NULLIFZERO(count_if(ic.hangup_cause = 'MEDIA_TIMEOUT'))::DECIMAL(10,2) as inbound_media_timeout
--    ARRAY_UNIQUE_AGG(case when ic.call_secs > 0 then ic.incoming_call_contact end) as call_contacts
FROM dev.data_science.incoming_calls ic
JOIN dev.data_science.user_profiles up ON ic.username = up.latest_username
WHERE
    -- DATE_UTC >= '2025-11-01'
    up.user_id_hex <> '000-00-000-000000000'
GROUP BY ALL

)
select 
    date_utc
    , user_id_hex
    , COALESCE(inbound_count, 0) AS inbound_count,
    COALESCE(inbound_duration_sec, 0) AS inbound_duration_sec,
    COALESCE(inbound_unanswered, 0) AS inbound_unanswered,
    COALESCE(inbound_count_over_15s, 0) AS inbound_count_over_15s,
    COALESCE(inbound_media_timeout, 0) AS inbound_media_timeout
FROM initial_table
""")
# user_features_inbound_calls_df.show()

#### messages

In [ ]:
user_features_messages_df = session.sql("""
WITH messages AS (
    SELECT
        date_utc,
        user_id_hex,
        "payload.message_direction" AS direction,
        CASE
            WHEN "payload.message_direction" = 'MESSAGE_DIRECTION_INBOUND'
                 AND "payload.origin"[0] = 'mailto:support@enflick.com' 
            THEN 'MESSAGE_TYPE_SYSTEM_TEXT'
            ELSE "payload.content_type"
        END AS message_type,
        "payload.routing_decision" AS routing_decision,
        -- single combo key for pivot
        "payload.message_direction" || '|' || 
        CASE
            WHEN "payload.message_direction" = 'MESSAGE_DIRECTION_INBOUND'
                 AND "payload.origin"[0] = 'mailto:support@enflick.com' 
            THEN 'MESSAGE_TYPE_SYSTEM_TEXT'
            ELSE "payload.content_type"
        END || '|' || "payload.routing_decision" AS combo,
        COUNT(*) AS message_count,
        ARRAY_UNION_AGG(
            IFF("payload.message_direction" = 'MESSAGE_DIRECTION_INBOUND',
                "payload.origin",
                "payload.target"
            )
        ) AS message_contacts
    FROM dev.data_science.MESSAGEDELIVERED
    WHERE
         NOT (
            instance_id LIKE 'TN_SERVER_%'
            AND "payload.content_type" IN (
                'MESSAGE_TYPE_TEXT',
                'MESSAGE_TYPE_IMAGE',
                'MESSAGE_TYPE_VIDEO',
                'MESSAGE_TYPE_AUDIO'
            )
        )
        AND user_id_hex IS NOT NULL
        -- AND DATE_UTC > '2025-11-10'
    GROUP BY ALL
    HAVING message_count > 0
)
, combined_key as (
select date_utc
    , user_id_hex
    , direction || '|' || message_type || '|'|| routing_decision as combined
    , sum(message_count) as ttl_message_count
FROM messages
GROUP BY 
    1, 2, 3
HAVING ttl_message_count > 0 
)
, pivoted_table as (
SELECT * 
FROM combined_key
PIVOT (
    SUM(ttl_message_count)
    FOR combined IN (
        'MESSAGE_DIRECTION_INBOUND|MESSAGE_TYPE_AUDIO|ROUTING_DECISION_ALLOW'   AS inbound_audio_allow, 
        'MESSAGE_DIRECTION_INBOUND|MESSAGE_TYPE_AUDIO|ROUTING_DECISION_DENY'    AS inbound_audio_deny, 
        'MESSAGE_DIRECTION_INBOUND|MESSAGE_TYPE_CONTACT|ROUTING_DECISION_ALLOW' AS inbound_contact_allow, 
        'MESSAGE_DIRECTION_INBOUND|MESSAGE_TYPE_CONTACT|ROUTING_DECISION_DENY'  AS inbound_contact_deny, 
        'MESSAGE_DIRECTION_INBOUND|MESSAGE_TYPE_IMAGE|ROUTING_DECISION_ALLOW'   AS inbound_image_allow, 
        'MESSAGE_DIRECTION_INBOUND|MESSAGE_TYPE_IMAGE|ROUTING_DECISION_DENY'    AS inbound_image_deny, 
        'MESSAGE_DIRECTION_INBOUND|MESSAGE_TYPE_MISSED_CALL|ROUTING_DECISION_UNKNOWN' AS inbound_missed_unk, 
        'MESSAGE_DIRECTION_INBOUND|MESSAGE_TYPE_PDF|ROUTING_DECISION_ALLOW'     AS inbound_pdf_allow, 
        'MESSAGE_DIRECTION_INBOUND|MESSAGE_TYPE_PDF|ROUTING_DECISION_DENY'      AS inbound_pdf_deny, 
        'MESSAGE_DIRECTION_INBOUND|MESSAGE_TYPE_SYSTEM_TEXT|ROUTING_DECISION_ALLOW' AS inbound_sys_allow, 
        'MESSAGE_DIRECTION_INBOUND|MESSAGE_TYPE_SYSTEM_TEXT|ROUTING_DECISION_DENY'  AS inbound_sys_deny, 
        'MESSAGE_DIRECTION_INBOUND|MESSAGE_TYPE_TEXT|ROUTING_DECISION_ALLOW'    AS inbound_text_allow, 
        'MESSAGE_DIRECTION_INBOUND|MESSAGE_TYPE_TEXT|ROUTING_DECISION_DENY'     AS inbound_text_deny, 
        'MESSAGE_DIRECTION_INBOUND|MESSAGE_TYPE_TEXT_FILE|ROUTING_DECISION_ALLOW' AS inbound_file_allow, 
        'MESSAGE_DIRECTION_INBOUND|MESSAGE_TYPE_TEXT_FILE|ROUTING_DECISION_DENY'  AS inbound_file_deny, 
        'MESSAGE_DIRECTION_INBOUND|MESSAGE_TYPE_UNKNOWN|ROUTING_DECISION_ALLOW' AS inbound_unk_allow, 
        'MESSAGE_DIRECTION_INBOUND|MESSAGE_TYPE_UNKNOWN|ROUTING_DECISION_DENY'  AS inbound_unk_demy, 
        'MESSAGE_DIRECTION_INBOUND|MESSAGE_TYPE_VIDEO|ROUTING_DECISION_ALLOW'   AS inbound_video_allow, 
        'MESSAGE_DIRECTION_INBOUND|MESSAGE_TYPE_VIDEO|ROUTING_DECISION_DENY'    AS inbound_video_deny, 
        'MESSAGE_DIRECTION_INBOUND|MESSAGE_TYPE_VOICEMAIL|ROUTING_DECISION_UNKNOWN' AS inbound_voicemail_unk, 
        'MESSAGE_DIRECTION_INBOUND|MESSAGE_TYPE_VOICEMAIL_TRANSCRIPT|ROUTING_DECISION_UNKNOWN' AS inbound_voicemailtr_unk, 
        'MESSAGE_DIRECTION_OUTBOUND|MESSAGE_TYPE_AUDIO|ROUTING_DECISION_ALLOW'  AS outbound_audio_allow, 
        'MESSAGE_DIRECTION_OUTBOUND|MESSAGE_TYPE_AUDIO|ROUTING_DECISION_DENY'   AS outbound_audio_deny, 
        'MESSAGE_DIRECTION_OUTBOUND|MESSAGE_TYPE_IMAGE|ROUTING_DECISION_ALLOW'  AS outbound_image_allow, 
        'MESSAGE_DIRECTION_OUTBOUND|MESSAGE_TYPE_IMAGE|ROUTING_DECISION_DENY'   AS outbound_image_deny, 
        'MESSAGE_DIRECTION_OUTBOUND|MESSAGE_TYPE_TEXT|ROUTING_DECISION_ALLOW'   AS outbound_text_allow, 
        'MESSAGE_DIRECTION_OUTBOUND|MESSAGE_TYPE_TEXT|ROUTING_DECISION_DENY'    AS outbound_text_deny, 
        'MESSAGE_DIRECTION_OUTBOUND|MESSAGE_TYPE_TEXT_FILE|ROUTING_DECISION_ALLOW' AS outbound_file_allow, 
        'MESSAGE_DIRECTION_OUTBOUND|MESSAGE_TYPE_UNKNOWN|ROUTING_DECISION_ALLOW' AS outbound_unk_allow, 
        'MESSAGE_DIRECTION_OUTBOUND|MESSAGE_TYPE_UNKNOWN|ROUTING_DECISION_DENY'  AS outbound_unk_deny, 
        'MESSAGE_DIRECTION_OUTBOUND|MESSAGE_TYPE_UNKNOWN|ROUTING_DECISION_UNKNOWN' AS outbound_unk_unk, 
        'MESSAGE_DIRECTION_OUTBOUND|MESSAGE_TYPE_VIDEO|ROUTING_DECISION_ALLOW'   AS outbound_video_allow, 
        'MESSAGE_DIRECTION_OUTBOUND|MESSAGE_TYPE_VIDEO|ROUTING_DECISION_DENY'    AS outbound_video_deny
    )
)
ORDER BY user_id_hex, date_utc
)
select date_utc
, user_id_hex
, COALESCE(inbound_audio_allow, 0) AS inbound_audio_allow,
COALESCE(inbound_audio_deny, 0) AS inbound_audio_deny,
COALESCE(inbound_contact_allow, 0) AS inbound_contact_allow,
COALESCE(inbound_contact_deny, 0) AS inbound_contact_deny,
COALESCE(inbound_image_allow, 0) AS inbound_image_allow,
COALESCE(inbound_image_deny, 0) AS inbound_image_deny,
COALESCE(inbound_missed_unk, 0) AS inbound_missed_unk,
COALESCE(inbound_pdf_allow, 0) AS inbound_pdf_allow,
COALESCE(inbound_pdf_deny, 0) AS inbound_pdf_deny,
COALESCE(inbound_sys_allow, 0) AS inbound_sys_allow,
COALESCE(inbound_sys_deny, 0) AS inbound_sys_deny,
COALESCE(inbound_text_allow, 0) AS inbound_text_allow,
COALESCE(inbound_text_deny, 0) AS inbound_text_deny,
COALESCE(inbound_file_allow, 0) AS inbound_file_allow,
COALESCE(inbound_file_deny, 0) AS inbound_file_deny,
COALESCE(inbound_unk_allow, 0) AS inbound_unk_allow,
COALESCE(inbound_unk_demy, 0) AS inbound_unk_demy,
COALESCE(inbound_video_allow, 0) AS inbound_video_allow,
COALESCE(inbound_video_deny, 0) AS inbound_video_deny,
COALESCE(inbound_voicemail_unk, 0) AS inbound_voicemail_unk,
COALESCE(inbound_voicemailtr_unk, 0) AS inbound_voicemailtr_unk,
COALESCE(outbound_audio_allow, 0) AS outbound_audio_allow,
COALESCE(outbound_audio_deny, 0) AS outbound_audio_deny,
COALESCE(outbound_image_allow, 0) AS outbound_image_allow,
COALESCE(outbound_image_deny, 0) AS outbound_image_deny,
COALESCE(outbound_text_allow, 0) AS outbound_text_allow,
COALESCE(outbound_text_deny, 0) AS outbound_text_deny,
COALESCE(outbound_file_allow, 0) AS outbound_file_allow,
COALESCE(outbound_unk_allow, 0) AS outbound_unk_allow,
COALESCE(outbound_unk_deny, 0) AS outbound_unk_deny,
COALESCE(outbound_unk_unk, 0) AS outbound_unk_unk,
COALESCE(outbound_video_allow, 0) AS outbound_video_allow,
COALESCE(outbound_video_deny, 0) AS outbound_video_deny
from pivoted_table

""")
# user_features_messages_df.show()

#### NPS rating

In [ ]:
user_features_nps_rating_df = session.sql("""
WITH initial_table as (
SELECT
    date_utc,
    user_id_hex,
    COUNT(call_rating)::FLOAT AS nps_call_rating_count,
    AVG(call_rating)::FLOAT AS nps_avg_call_rating,
    MAX(call_rating)::FLOAT AS nps_max_call_rating,
    MIN(call_rating)::FLOAT AS nps_min_call_rating,
FROM
    dev.data_science.call_ratings_combined_sources
WHERE
    call_rating > 0
    AND user_id_hex <> '000-00-000-000000000'
GROUP BY ALL
)
select 
    date_utc
    , user_id_hex
    , COALESCE(nps_call_rating_count, 0) nps_call_rating_count
    , COALESCE(nps_avg_call_rating, 0) nps_avg_call_rating
    , COALESCE(nps_max_call_rating, 0) nps_max_call_rating
    , COALESCE(nps_min_call_rating, 0) nps_min_call_rating
FROM initial_table
""")
# user_features_nps_rating_df.show()

#### outbound calls

In [ ]:
user_features_outbound_calls_df = session.sql("""
WITH initial_table as (
SELECT
    oc.date_utc,
    up.user_id_hex,
    COUNT(1)::DECIMAL(10,2) as count,
    SUM(oc.call_secs::DECIMAL(10,2)) as duration_sec,
    NULLIFZERO(count_if(oc.call_secs = 0))::DECIMAL(10,2) as unanswered,
    NULLIFZERO(count_if(oc.call_secs > 15))::DECIMAL(10,2) as count_over_15s,
    NULLIFZERO(count_if(oc.hangup_cause = 'MEDIA_TIMEOUT'))::DECIMAL(10,2) as media_timeout,
FROM dev.data_science.outgoing_calls oc
JOIN dev.data_science.user_profiles up ON oc.username = up.latest_username
WHERE
    up.user_id_hex <> '000-00-000-000000000'
    -- AND date_utc > '2025-11-01'
GROUP BY ALL
)
select 
    date_utc
    , user_id_hex,
COALESCE(count, 0) AS count,
COALESCE(duration_sec, 0) AS duration_sec,
COALESCE(unanswered, 0) AS unanswered,
COALESCE(count_over_15s, 0) AS count_over_15s,
COALESCE(media_timeout, 0) AS media_timeout
FROM initial_table
""")
# user_features_outbound_calls_df.show()

#### Redial

In [ ]:
user_features_redial_df = session.sql("""

WITH redials AS (
    SELECT
        username,
        created_at,
        call_direction,
        from_contact_value,
        to_contact_value,
        callduration,
        client_type,
        lag(client_type) OVER (PARTITION BY username ORDER BY created_at) AS prev_client_type,
        lag(callduration) OVER (PARTITION BY username ORDER BY created_at) AS prev_call_duration,
        lag(from_contact_value) OVER (PARTITION BY username ORDER BY created_at) AS prev_from_contact,
        lag(to_contact_value) OVER (PARTITION BY username ORDER BY created_at) AS prev_to_contact,
        (
            created_at < dateadd(second, nvl(prev_call_duration, 0) + 60, lag(created_at) OVER (PARTITION BY username ORDER BY created_at))
            AND prev_call_duration > 5
            AND call_direction in ('outgoing', 'incoming')
            AND (
                (from_contact_value = prev_from_contact AND to_contact_value = prev_to_contact) or 
                (from_contact_value = prev_to_contact AND to_contact_value = prev_from_contact)
            )
        ) AS redial
    FROM   dev.data_science.ONCALLEND_CALLBACKS
    -- WHERE created_at > '2025-11-01'
)
, test as (
SELECT
    created_at::DATE AS date_utc,
    user_id_hex,
    COALESCE(client_type, prev_client_type) AS client_type_redials,
    count_if(redial) AS feature_value_numeric
FROM redials
JOIN dev.data_science.user_profiles up on username = latest_username
GROUP BY ALL
)
, pivot_table as (
select *
from test
PIVOT (
        MAX(feature_value_numeric)
        FOR client_type_redials IN (
        'TN_IOS_FREE' as tn_ios_free
        ,null as null_name 
        ,'TN_ANDROID' as tn_android
        ,'2L_ANDROID' as android
        , 'TN_WEB' as web)
    ) AS p

    )
select date_utc
    , user_id_hex,
COALESCE(tn_ios_free, 0) AS tn_ios_free,
COALESCE(null_name, 0) AS null_name,
COALESCE(tn_android, 0) AS tn_android,
COALESCE(android, 0) AS android,
COALESCE(web, 0) AS web
FROM pivot_table

""")
# user_features_redial_df.show()

#### sessions

In [ ]:
user_features_sessions_df = session.sql("""
WITH inital_table AS (
  SELECT 
      m.date_utc,
      up.user_id_hex,
      SUM(m.time_in_app_mins_per_day::DECIMAL(18,4)) AS time_in_app_mins,
      DATEDIFF(day, MIN(up.registered_at), m.date_utc)::DECIMAL(18,0) AS tenure_days,
      SUM(m.num_sessions::DECIMAL(18,0)) AS session_count
  FROM dev.data_science.metrics_daily_userlevel_app_time_sessions m 
  JOIN dev.data_science.user_profiles up
    ON m.username = up.latest_username
  WHERE up.user_id_hex <> '000-00-000-000000000'
    AND m.client_type IN ('TN_IOS_FREE', 'TN_ANDROID')
  GROUP BY ALL
),
features AS (
  SELECT
    date_utc,
    user_id_hex,
    COALESCE(time_in_app_mins, 0) AS time_in_app_mins,
    COALESCE(tenure_days, 0)      AS tenure_days,
    COALESCE(session_count, 0)    AS session_count,
    IFF(COALESCE(time_in_app_mins, 0) > 1, 1, 0) AS is_active_day
  FROM inital_table
),
rollups AS (
  SELECT
    date_utc,
    user_id_hex,
    time_in_app_mins,
    tenure_days,
    session_count,

    COALESCE(
      SUM(is_active_day) OVER (
        PARTITION BY user_id_hex
        ORDER BY date_utc
        RANGE BETWEEN INTERVAL '7 DAYS' PRECEDING AND INTERVAL '1 DAY' PRECEDING
      ),
      0
    ) AS active_days_prev_7,

    COALESCE(
      SUM(is_active_day) OVER (
        PARTITION BY user_id_hex
        ORDER BY date_utc
        RANGE BETWEEN INTERVAL '14 DAYS' PRECEDING AND INTERVAL '1 DAY' PRECEDING
      ),
      0
    ) AS active_days_prev_14,

    COALESCE(
      SUM(is_active_day) OVER (
        PARTITION BY user_id_hex
        ORDER BY date_utc
        RANGE BETWEEN INTERVAL '30 DAYS' PRECEDING AND INTERVAL '1 DAY' PRECEDING
      ),
      0
    ) AS active_days_prev_30,

    COALESCE(
      SUM(is_active_day) OVER (
        PARTITION BY user_id_hex
        ORDER BY date_utc
        RANGE BETWEEN INTERVAL '90 DAYS' PRECEDING AND INTERVAL '1 DAY' PRECEDING
      ),
      0
    ) AS active_days_prev_90

  FROM features
)
SELECT
  date_utc,
  user_id_hex,
  time_in_app_mins,
  tenure_days,
  session_count,
  active_days_prev_7,
  active_days_prev_14,
  active_days_prev_30,
  active_days_prev_90
FROM rollups
""")
# user_features_sessions_df.show()

## Feature store

#### Create FS

In [ ]:
# fs = FeatureStore(
#     session=session,
#     database="dev",
#     name="user_activity_feature_store",
#     default_warehouse="ds_wh_medium",
#     creation_mode=CreationMode.CREATE_IF_NOT_EXIST
# )

#### Connect to FS

In [ ]:
fs = FeatureStore(
    session=session,
    database="dev",
    name="user_activity_feature_store",
    default_warehouse="analytics_wh_flex"
)

#### Create and register entities

In [ ]:
entity = Entity(
    name="user_entity_v2",
    join_keys=["user_id_hex", "date_utc"],
    desc="user entity"
)
fs.register_entity(entity)

In [ ]:
#### Get existing entities

In [ ]:
entity = fs.get_entity("user_entity_v2")

#### Create and register feature views

In [ ]:
# Ads
user_features_ads_fv = FeatureView(
    name="user_features_ads",
    entities=[entity],
    feature_df=user_features_ads_df,
    timestamp_col = "date_utc",
    refresh_freq="24 hours",
    refresh_mode = "incremental",
    desc="features about user ads"
)

fs.register_feature_view(
    feature_view=user_features_ads_fv,
    version="1.2",
)

In [ ]:
# Call quality
user_features_call_quality_fv = FeatureView(
    name="user_features_call_quality",
    entities=[entity],
    feature_df=user_features_call_quality_df,
    timestamp_col = "date_utc",
    refresh_freq="24 hours",
    refresh_mode = "incremental",
    desc="features about user call quality"
)

fs.register_feature_view(
    feature_view=user_features_call_quality_fv,
    # version="4"
    version="1.2"
)

In [ ]:
# Call rating
user_features_call_rating_fv = FeatureView(
    name="user_features_call_rating",
    entities=[entity],
    feature_df=user_features_call_rating_df,
    timestamp_col = "date_utc",
    refresh_freq="24 hours",
    refresh_mode = "incremental",
    desc="features about user call rating"
)

fs.register_feature_view(
    feature_view=user_features_call_rating_fv,
    # version="4"
    version="1.2"
)

In [ ]:
# Data usage
user_features_data_usage_fv = FeatureView(
    name="user_features_data_usage",
    entities=[entity],
    feature_df=user_features_data_usage_df,
    timestamp_col = "date_utc",
    refresh_freq="24 hours",
    refresh_mode = "incremental",
    desc="features about user data usage"
)

fs.register_feature_view(
    feature_view=user_features_data_usage_fv,
    version="1.2"
)

In [ ]:
# Drawer event
user_features_drawer_event_fv = FeatureView(
    name="user_features_drawer_event",
    entities=[entity],
    feature_df=user_features_drawer_event_df,
    timestamp_col = "date_utc",
    refresh_freq="24 hours",
    refresh_mode = "incremental",
    desc="features about user drawer event"
)

fs.register_feature_view(
    feature_view=user_features_drawer_event_fv,
    version="1.2"
)

In [ ]:
# Inbound calls
user_features_inbound_calls_fv = FeatureView(
    name="user_features_inbound_calls",
    entities=[entity],
    feature_df=user_features_inbound_calls_df,
    timestamp_col = "date_utc",
    refresh_freq="24 hours",
    refresh_mode = "incremental",
    desc="features about user inbound calls"
)

fs.register_feature_view(
    feature_view=user_features_inbound_calls_fv,
    version="1.2"
)

In [ ]:
# Messages
user_features_messages_fv = FeatureView(
    name="user_features_messages",
    entities=[entity],
    feature_df=user_features_messages_df,
    timestamp_col = "date_utc",
    refresh_freq="24 hours",
    refresh_mode = "incremental",
    desc="features about user messages"
)

fs.register_feature_view(
    feature_view=user_features_messages_fv,
    version="1.2"
)

#user_features_messages_df

In [ ]:
# NPS Rating
user_features_nps_rating_fv = FeatureView(
    name="user_features_nps_rating",
    entities=[entity],
    feature_df=user_features_nps_rating_df,
    timestamp_col = "date_utc",
    refresh_freq="24 hours",
    refresh_mode = "incremental",
    desc="features about user NPS Rating"
)

fs.register_feature_view(
    feature_view=user_features_nps_rating_fv,
    version="1.2"
)

In [ ]:
# Outbound calls
user_features_outbound_calls_fv = FeatureView(
    name="user_features_outbound_calls",
    entities=[entity],
    feature_df=user_features_outbound_calls_df,
    timestamp_col = "date_utc",
    refresh_freq="24 hours",
    refresh_mode = "incremental",
    desc="features about user outbound calls"
)

fs.register_feature_view(
    feature_view=user_features_outbound_calls_fv,
    version="1.2"
)

In [ ]:
# Redial
user_features_redial_fv = FeatureView(
    name="user_features_redial",
    entities=[entity],
    feature_df=user_features_redial_df,
    timestamp_col = "date_utc",
    refresh_freq="24 hours",
    refresh_mode = "incremental",
    desc="features about user redial"
)

fs.register_feature_view(
    feature_view=user_features_redial_fv,
    version="1.2"
)

In [ ]:
# sessions
user_features_sessions_fv = FeatureView(
    name="user_features_sessions",
    entities=[entity],
    feature_df=user_features_sessions_df,
    timestamp_col = "date_utc",
    refresh_freq="24 hours",
    refresh_mode = "incremental",
    desc="features about user sessions"
)

fs.register_feature_view(
    feature_view=user_features_sessions_fv,
    version="1.2"
)

## Done